# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233979910593                   -0.50    7.0         
  2   -7.250043375474       -1.79       -1.39    1.0   7.73ms
  3   -7.250813159294       -3.11       -1.83    2.0   8.56ms
  4   -7.250896674667       -4.08       -1.84    2.0   9.13ms
  5   -7.251336342418       -3.36       -2.92    1.0   7.74ms
  6   -7.251338495578       -5.67       -3.31    2.0   9.51ms
  7   -7.251338728054       -6.63       -3.64    1.0   7.72ms
  8   -7.251338788659       -7.22       -4.15    1.0   7.72ms
  9   -7.251338797954       -8.03       -4.58    2.0   9.52ms
 10   -7.251338798634       -9.17       -5.28    1.0   7.89ms
 11   -7.251338798696      -10.21       -5.61    2.0   9.67ms
 12   -7.251338798703      -11.15       -6.03    1.0   8.00ms
 13   -7.251338798704      -11.91       -6.40    2.0   9.51ms
 14   -7.251338798705      -12.68       -6.91    1.0   7.97ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05047212515354877
[ Info: Arnoldi iteration step 2: normres = 0.47718420212937623
[ Info: Arnoldi iteration step 3: normres = 0.647138414325675
[ Info: Arnoldi iteration step 4: normres = 0.5621855772447031
[ Info: Arnoldi iteration step 5: normres = 0.4124645267185201
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.05e-01, 6.17e-02, 2.91e-01, 2.65e-01, 1.95e-02)
[ Info: Arnoldi iteration step 6: normres = 0.36206976692753545
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.10e-02, 8.13e-02, 3.25e-01, 8.71e-02, 9.93e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0824942326276317
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.28e-03, 9.50e-03, 2.22e-02, 2.87e-02, 5.89e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12152076542696987
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.78e-0